In [1]:
!pip install gradio transformers accelerate peft --quiet

In [2]:
!pip install --upgrade transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from peft import PeftModel
import gradio as gr

# Load LoRA model using the approach from model_evaluation_and_finetuning.ipynb
model_name = "./lora_academic_model"
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
model = PeftModel.from_pretrained(base_model, model_name).to(device)
model.eval()

# Add a padding token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Test the model before running Gradio
test_prompt = "What is the capital of France?"
inputs = tokenizer(test_prompt, return_tensors="pt").to(device)
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=32)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Test response:", response)

# Inference function for Gradio
def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=128)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Gradio UI
def gradio_interface():
    gr.Interface(
        fn=generate_response,
        inputs=gr.Textbox(lines=2, placeholder="Enter your prompt here..."),
        outputs=gr.Textbox(),
        title="NLP Model Demo",
        description="Enter a prompt to interact with the model."
    ).launch()

gradio_interface()

In [ ]:
# Inference function for Gradio
def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=128)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
# Gradio UI
def gradio_interface():
    gr.Interface(
        fn=generate_response,
        inputs=gr.Textbox(lines=2, placeholder="Enter your prompt here..."),
        outputs=gr.Textbox(),
        title="NLP Model Demo",
        description="Enter a prompt to interact with the model."
    ).launch()

gradio_interface()

In [ ]:
# To run Gradio, just execute the cell above. No need for a separate command.